### Download dependenices

In [2]:
from shapely.geometry import Polygon, mapping
import json

from sentinelhub import (
    SentinelHubRequest,
    DataCollection,
    MimeType,
    bbox_to_dimensions,
    SHConfig,
    BBox,
    CRS,
)
import geopandas as gpd

### Make signin

In [3]:
config = SHConfig()
config.sh_client_id = "eab392d1-68dc-429d-a416-977e7ce3c2ce"
config.sh_client_secret = "dxWUQMgvtjjNkn9sHkaceFD0DUZJf0jT"

In [4]:
gdf = gpd.read_file("dataset/l2022_survey_agri_toscana.gpkg")

In [5]:
wheat  = gdf[gdf["surveycprnlc"] == "B12"]
# Get 

In [6]:
# Test geometry get the first geometry Polygon
polygon = wheat.geometry.iloc[0]

In [7]:
bbox = BBox(bbox=polygon.bounds, crs=CRS.WGS84)

In [8]:
bbox

BBox(((15.850925989328346, 40.89044511677216), (15.851334785432137, 40.89086189898945)), crs=CRS('4326'))

In [55]:


evalscript = """
//VERSION=3

function setup() {
    return {
        input: [{
            datasource: "S2L2A", 
            bands: ["B02", "B03", "B04", "B08", "B05", "B06", "B07", "B8A", "B11", "B12", "B01", "B09", "CLM"],
            units: "REFLECTANCE"
        },
        {
            datasource: "L8L1C", 
            bands: ["B02", "B03", "B04", "B05", "B06", "B07", "B08", "CLM"],
            units: "REFLECTANCE"
        }],
        output: {
            id: "default",
            bands: 19,
            sampleType: "FLOAT32"
        }
    };
}

function evaluatePixel(samples) {
    let S2 = samples.S2L2A;
    let L8 = samples.L8L1C;
    
    let cloud_mask = S2.CLM || L8.CLM;

    if (cloud_mask == 1) {
        return Array(19).fill(NaN);
    }
    
    return [
        S2.B02, S2.B03, S2.B04, S2.B08, S2.B05, S2.B06, S2.B07, S2.B8A, S2.B11, S2.B12, S2.B01, S2.B09,
        L8.B02, L8.B03, L8.B04, L8.B05, L8.B06, L8.B07, L8.B08
    ];
}
"""

request = SentinelHubRequest(
    evalscript=evalscript,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A,
            time_interval=("2021-09-01", "2023-04-01"),
            mosaicking_order="leastCC",
        ),
        SentinelHubRequest.input_data(
            data_collection=DataCollection.HARMONIZED_LANDSAT_SENTINEL,
            time_interval=("2021-09-01", "2023-04-01"),
            mosaicking_order="leastCC",
        ),
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=bbox,
    size=bbox_to_dimensions(bbox, resolution=10),
    config=config,
    data_folder="dimages",
)

In [56]:
response = request.get_data(save_data=True)

DownloadFailedException: Failed to download from:
https://services.sentinel-hub.com/api/v1/process
with HTTPError:
400 Client Error: Bad Request for url: https://services.sentinel-hub.com/api/v1/process
Server response: "{"status": 400, "reason": "Bad Request", "message": "Dataset with id: S2L2A not found.", "code": "RENDERER_EXCEPTION"}"